In [87]:
# import libraries
import pandas as pd
import numpy as np
from scipy.stats import linregress
import matplotlib.pyplot as plt

## Cleaning Natural Gas Data

In [88]:
# import raw gas usage data from csv
file_path = 'raw_data/10yr_gas_data.csv'
df = pd.read_csv(file_path)

In [89]:
# removing individual date columns and adding a new date column that includes all 10 years
dates = []
states = []
values = []

# loop through columns of columns (DC is included so there are 51 state columns)
for i in range(1,52):

   #loop through rows
    for j in range(0,132): 
        date = df.iloc[j,0] # gets date from first column
        dates.append(date)  # adds date to dates list

        state_str = df.columns[i] # gets column title
        state_str = state_str.split(' Natural', 1) #splits the title at the first occurence of the word 'natural'
        state = state_str[0] # saves the state name 
        states.append(state) # adds the state to the states list

        value = df.iloc[j,i] # retrieves the value from the current row,value pair
        values.append(value) # adds the value to the values list 

# create data frame to store gas data
df_clean = pd.DataFrame({
    'Date': dates,
    'State': states,
    'Natural Gas Consumption (MMcf)':values
})

df_clean.head()

,Date,State,Natural Gas Consumption (MMcf)
0,Jan-2010,Alabama,9202.647
1,Feb-2010,Alabama,7762.261
2,Mar-2010,Alabama,6612.051
3,Apr-2010,Alabama,2987.217
4,May-2010,Alabama,1142.784


In [90]:
# output clean gas data to csv
output_path = 'clean_data/natural_gas_data.csv'
df_clean.to_csv(output_path)

## Cleaning Temperature Data

In [91]:
# read in temp data from csv into dataframe
file_path = 'raw_data/all_states_ave_temp.csv'
df2 = pd.read_csv(file_path)

In [92]:
# drop NaN values from the temp dataframe
df2.dropna(how='any', inplace=True)

In [93]:
# list of columns to rename
column_names = {
    'Unnamed: 1': 'State', 
    'Average Temperature': 'Avg Temp (F)',
    'Unnamed: 3': 'Anomaly',
    'Alabama' : 'Date'
    }
# renaming columns
df2.rename(columns=column_names, inplace=True)
#reseting index
df2.reset_index(drop=True, inplace=True)
# showing results
df2.head()

,Date,State,Average Temperature,Anomaly
0,200001,Alabama,46.5,1.6
1,200002,Alabama,52.3,2.5
2,200003,Alabama,58.9,2.7
3,200004,Alabama,60.1,-3.8
4,200005,Alabama,74,2.4


In [94]:
# output the clean temp data to csv file
output_path = 'clean_data/avg_temp_by_state.csv'
df2.to_csv(output_path)

## Cleaning Population Data

In [95]:
# read pop data from csv
pop_file = 'clean_data/10yr_population_data.csv'
pop_df = pd.read_csv(pop_file)

In [96]:
# drop unnamed index colum that is added
pop_df.drop(columns='Unnamed: 0', inplace=True)

In [97]:
# loops through and removes '.' from  state string
for i in range(0,len(pop_df)):
    st_string = pop_df.iloc[i, 0]
    st_string = st_string.split(".", 2) 
    pop_df.iloc[i, 0] = st_string[0]

In [98]:
# drop DC from population data to match rest of data
pop_df.drop(pop_df.loc[pop_df['state'] == 'District of Columbia'].index, inplace=True)

In [99]:
# reformat pop data to match the layout of other data (each year was a column, now theres one year column)
states = []
years = []
pops = []

# loop through columns (10 years)
for i in range(1, 11): 
    # loop through rows (49 states)
    for j in range(0, 50):
        state = pop_df.iloc[j,0]
        states.append(state)

        year = pop_df.columns[i]
        years.append(year)

        pop = pop_df.iloc[j, i]
        pops.append(pop)

# create new data frame for reformatted pop data
pop_df = pd.DataFrame({
    'State': states,
    'Year': years,
    'Population': pops
})

pop_df.head()

,State,Year,Population
0,Alabama,2010,"4,785,437"
1,Alaska,2010,"713,910"
2,Arizona,2010,"6,407,172"
3,Arkansas,2010,"2,921,964"
4,California,2010,"37,319,502"


In [100]:
# sort states alphabetically and reset index
pop_df.sort_values(by='State', inplace=True)
pop_df.reset_index(drop=True, inplace=True)

In [101]:
# output the clean temp data to csv file
output_path = 'clean_data/pop_data.csv'
pop_df.to_csv(output_path)

## Data Merging and Filtering

In [102]:
# file paths
gas_file = 'clean_data/natural_gas_data.csv'
weather_file = 'clean_data/avg_temp_by_state.csv'
pop_file = 'clean_data/pop_data.csv'

# import the files 
weather_df = pd.read_csv(weather_file)
gas_df = pd.read_csv(gas_file)
pop_df = pd.read_csv(pop_file)

In [103]:
# droping the extra index column that gets added when importing
gas_df.drop(columns='Unnamed: 0', inplace=True)
weather_df.drop(columns='Unnamed: 0', inplace=True)
pop_df.drop(columns='Unnamed: 0', inplace=True)

In [104]:
# drop dates before 2010 and after 2019 from weather data
weather_df.drop(weather_df.loc[weather_df['Date'] < 201000].index, inplace=True)
weather_df.drop(weather_df.loc[weather_df['Date'] >= 202000].index, inplace=True)

In [105]:
# drop 2020 (since its not in the weather data) as well as DC (for the same reason)
# hawaii also had to be dropped from gas, as the data was not available via noaa
gas_df.drop(gas_df.loc[gas_df['Date'].str.contains("2020")].index, inplace=True)
gas_df.drop(gas_df.loc[gas_df['State']=='District of Columbia'].index, inplace=True)
gas_df.drop(gas_df.loc[gas_df['State']=='Hawaii'].index, inplace=True)

In [106]:
# format the date variables to be uniform for merging - WEATHER
weather_df['Date'] = pd.to_datetime(weather_df['Date'], format='%Y%m')
weather_df['Date'] = weather_df['Date'].apply(lambda x: x.strftime('%Y-%m'))

In [107]:
# format the date variables to be uniform for merging - GAS
gas_df['Date'] = pd.to_datetime(gas_df['Date'], format='%b-%Y')
gas_df['Date'] = gas_df['Date'].apply(lambda x: x.strftime('%Y-%m'))

In [108]:
# merge the weather and gas data together into single df and display results
combined_df = pd.merge(gas_df, weather_df, how='outer')
combined_df.to_csv('clean_data/combined_gas_and_temp.csv')

In [109]:
# convert date strings to datetime objects for easier referencing
combined_df['Date'] = pd.to_datetime(combined_df['Date'], format='%Y-%m')
pop_df['Date'] = pd.to_datetime(pop_df['Year'], format="%Y")
pop_df.drop(columns='Year', inplace=True)

In [110]:
# group combined weather and gas data by State and Year
combined_year_df = combined_df.groupby([combined_df['State'],combined_df['Date'].dt.year])

In [111]:
# find the average values for each year and reset the index
combined_means = combined_year_df.mean()
combined_means = combined_means.reset_index()

In [112]:
# convert date string to datetime object for merge
combined_means['Date'] = pd.to_datetime(combined_means['Date'], format='%Y')

In [113]:
# add population data with weather and gas averages into new DF
combo_years = pd.merge(combined_means, pop_df, how='left', on=['Date', 'State'])

In [114]:
# drop NaN values
combo_years.dropna(how='any', inplace=True)
# convert datetime object back into 'Year' string 
combo_years['Date'] = combo_years['Date'].apply(lambda x: x.strftime('%Y'))
# rename column
combo_years.rename(columns={'Date':'Year'}, inplace=True)
#reset index
combo_years.reset_index(drop=True, inplace=True)

In [115]:
combo_years.head()

,State,Year,Natural Gas Consumption (MMcf),Average Temperature,Anomaly,Population
0,Alabama,2010,3517.939750,62.708333,-1.425000,"4,785,437"
1,Alabama,2011,3048.491417,63.833333,-0.300000,"4,799,069"
2,Alabama,2012,2298.298333,64.916667,0.783333,"4,815,588"
3,Alabama,2013,2921.568667,62.600000,-1.533333,"4,830,081"
4,Alabama,2014,3250.500833,62.033333,-2.100000,"4,841,799"


In [116]:
# remove commas from population strings and then convert column to integers
for i in range(0, len(combo_years)):
   combo_years.iloc[i, 5] = int(combo_years.iloc[i, 5].replace(',',''))

combo_years['Population'] = combo_years['Population'].astype(int)
combo_years.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 490 entries, 0 to 489
Data columns (total 6 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   State                           490 non-null    object 
 1   Year                            490 non-null    object 
 2   Natural Gas Consumption (MMcf)  490 non-null    float64
 3    Average Temperature            490 non-null    float64
 4   Anomaly                         490 non-null    float64
 5   Population                      490 non-null    int32  
dtypes: float64(3), int32(1), object(2)
memory usage: 21.2+ KB


In [117]:
# drop states that use less than 5000 MMcf per year
combo_years = combo_years.loc[combo_years['Natural Gas Consumption (MMcf)'] >= 5000]
combo_years = combo_years.reset_index(drop=True)
len(combo_years['State'].unique())

24

In [118]:
# add a average gas usage per person column
combo_years['Gas Usage Per Person (MMcf)'] = combo_years['Natural Gas Consumption (MMcf)'] / combo_years['Population']
combo_years['Gas Usage Per Person (MMcf)'] = combo_years['Gas Usage Per Person (MMcf)'].astype(float)

,Natural Gas Consumption (MMcf),Average Temperature,Anomaly,Population,Gas Usage Per Person (MMcf)
State,,,,,
California,37477.004692,59.520000,-0.013333,3.861819e+07,0.000972
Colorado,10648.595550,46.613333,-0.011667,5.403199e+06,0.001974
Georgia,10060.952758,64.768333,-0.040000,1.014747e+07,0.000992
Illinois,34740.719725,52.895833,-0.045833,1.282232e+07,0.002710
Indiana,11300.906708,52.691667,-0.066667,6.603554e+06,0.001711
Iowa,5659.685417,48.112037,-0.429630,3.113000e+06,0.001819
Kansas,5653.002736,54.383333,-1.091667,2.890962e+06,0.001955
Maryland,6740.274267,55.923333,-0.051667,5.948976e+06,0.001133
Massachusetts,10326.958658,49.424167,-0.050833,6.757157e+06,0.001528


In [119]:
combo_years.to_csv('clean_data/combined_w_pop.csv')